In [36]:
# activate project environment
# include these lines of code in any future scripts/notebooks
#---
import Pkg
if !haskey(Pkg.installed(), "AA228FinalProject")
    jenv = joinpath(dirname(@__FILE__()), ".") # this assumes the notebook is in the same dir
    # as the Project.toml file, which should be in top level dir of the project. 
    # Change accordingly if this is not the case.
    Pkg.activate(jenv)
end
#---

"/Users/allisonlettiere/Desktop/CS238FinalProject/Project.toml"

In [37]:
# import necessary packages
using AA228FinalProject
using POMDPs
using POMDPPolicies
using POMDPModels
using POMDPModelTools
using BeliefUpdaters
using ParticleFilters
using POMDPSimulators
using Cairo
using Gtk
using Random
using Printf

In [38]:
sensor = Bumper()
config = 3 # 1,2, or 3

num_x_pts = 50
num_y_pts = 50
num_th_pts = 20
sspace = DiscreteRoombaStateSpace(num_x_pts,num_y_pts,num_th_pts)

vlist = [0,1,10]
omlist = [-1,0,1]
aspace = vec(collect(RoombaAct(v, om) for v in vlist, om in omlist))

m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config, sspace=sspace, aspace=aspace))

RoombaPOMDP{Bumper,Bool}(Bumper(), RoombaMDP{DiscreteRoombaStateSpace,Array{RoombaAct,1}}
  v_max: Float64 10.0
  om_max: Float64 1.0
  dt: Float64 0.5
  contact_pen: Float64 -1.0
  time_pen: Float64 -0.1
  goal_reward: Float64 10.0
  stairs_penalty: Float64 -10.0
  config: Int64 3
  room: AA228FinalProject.Room
  sspace: DiscreteRoombaStateSpace
  aspace: Array{RoombaAct}((9,))
  _amap: Dict{RoombaAct,Int64}
)

In [39]:
num_particles = 2000
resampler = BumperResampler(num_particles)

spf = SimpleParticleFilter(m, resampler)

v_noise_coefficient = 2.0
om_noise_coefficient = 0.5

belief_updater = RoombaParticleFilter(spf, v_noise_coefficient, om_noise_coefficient);

## Policy

randomized point-based value iteration (Algorithm 6.5 )

In [44]:
function sumNextStateObservation(state, action, alphaAO)
    sum = 0
    e, gr, gwn, sr, swn, next_x, next_y, next_th, next_status, sp = checkCollision(state, action)
    
    ##0 -- does not hit wall, 1 -- hits goal zone, 2 -- hits stairs, 3 -- hits other wall
    observations = [0, 1, 2, 3]
    observationFunction = 0
    ob = 1
    for o in observations
        if o == 0
            if(!wall_contact(e, s))
                observationFunction = 1
                ob = 0
            end
        elseif o == 1
            if(contact_wall(gr, gwn, [next_x, next_y]))  
                observationFunction = 1
            end
        elseif o == 2
            if(contact_wall(sr, swn, [next_x, next_y]))  
                observationFunction = 1
            end
        else
            if(wall_contact(e, s))
                observationFunction = 1
            end
        end

        transitionFunction = 1 ##if reachable
        sum += observationFunction * transitionFunction * alphaAO[action, ob + 1]
    end
    return sum
end

sumNextStateObservation (generic function with 1 method)

In [45]:
function checkCollision(state, action)
    e = mdp(m)
    v, om = action
    v = clamp(v, 0.0, e.v_max)
    om = clamp(om, -e.om_max, e.om_max)

    # propagate dynamics without wall considerations
    x, y, th, _ = s
    dt = e.dt

    # dynamics assume robot rotates and then translates
    next_th = wrap_to_pi(th + om*dt)

    # make sure we arent going through a wall
    p0 = SVector(x, y)
    heading = SVector(cos(next_th), sin(next_th))
    des_step = v*dt
    next_x, next_y = legal_translate(e.room, p0, heading, des_step)

    # Determine whether goal state or stairs have been reached
    grn = mdp(m).room.goal_rect
    gwn = mdp(m).room.goal_wall
    srn = mdp(m).room.stair_rect
    swn = mdp(m).room.stair_wall
    gr = mdp(m).room.rectangles[grn]
    sr = mdp(m).room.rectangles[srn]    
    
    next_status = 1.0*contact_wall(gr, gwn, [next_x, next_y]) - 1.0*contact_wall(sr, swn, [next_x, next_y])
    sp = RoombaState(next_x, next_y, next_th, next_status)
    
    return e, gr, gwn, sr, swn, next_x, next_y, next_th, next_status, sp
end

checkCollision (generic function with 1 method)

In [47]:
function backupBelief(b, gamma)
    O = [0, 1]
    numberOfStates = len(sspace)
    numberOfAction = len(aspace)
    alphaAO = ones(numberOfActions, 2)
    
    for action in aspace
        for observation in O
            bp = POMDPs.update(belief_updater, b, action, observation)
            maxTranspose = 0
            for a in gamma
                t = a'bp
                if (t > maxTranspose)
                    maxTranspose = t
                    alphaAO[action, observation + 1] = a
                end
            end
        end 
        
        numberOfStates = len(sspace)
        numberOfAction = len(aspace)
        alpha = ones(numberOfStates, numberOfActions)
        
        e, gr, gwn, sr, swn, next_x, next_y, next_th, next_status = checkCollision(state, action)
        sp = RoombaState(next_x, next_y, next_th, next_status)
        
        for state in sspace            
            alpha[state, action] = POMDPs.reward(state, action, sp) + discountFactor * sumNextStateObservation(state, action, alphaAO)
        end
    end
    newAlpha = 0
    maxAlphaTranspose = 0
    columns = [alpha[:,i] for i in 1:size(alpha,2)]
    for a in columns
        aT = a'b
        if(aT > maxAlphaTranspose)
            maxAlphaTranspose = aT
            newAlpha = a
        end
    end
    return newAlpha
end     

backupBelief (generic function with 1 method)

In [50]:
function RandomizedPointBasedUpdate(B, gamma)
    BP = B
    gammaP = []
    print(BP)
    while !(isempty(BP))
        b = rand(BP)
        filter!(e->e != b,BP)
        alphaVector = backupBelief(b, gamma)
        maxU = 0
        oldMaxA = 0
        for alpha in gamma
            t = alpha'b
            if t>maxU
                maxU = t
                oldMaxA = alpha
            end
        end
        oldUtility = oldMaxA'b
        newUtility = alphaVector'b
        if (newUtility >= oldUtility)
            push!(gammaP, alphaVector)
        else
            push!(gammaP, oldMaxA)
        end
    end
    return gammaP
end

RandomizedPointBasedUpdate (generic function with 1 method)

In [51]:
discountFactor = 0.9

goal_xy = get_goal_xy(m)
goal_x, goal_y = goal_xy

RW = 5

if config == 1
    x = 10+RW 
    y = -RW
elseif config== 2
    x = -20+RW
    y = RW
else
    x = -20+RW
    y = RW
end

ang_to_goal = atan(goal_y - y, goal_x - x)
si = RoombaState(x,y,ang_to_goal,0.)
sf = RoombaState(goal_x,goal_y, 1, 0.)
action = (5, ang_to_goal)

#startingReward = reward(m, si, action, sf)
startingReward = 5
firstAlphaVector = (1/(1-discountFactor)) * startingReward
gamma = []
push!(gamma, firstAlphaVector)

B = []
for i in 100:
    b = ParticleCollection{RoombaState}
    push!(B, b)
    ##change pomdp
    resample(up.spf.resample, d, up.spf.rng)

for i in 1:10    
    gamma = RandomizedPointBasedUpdate(B, gamma)
end

ParticleCollection{RoombaState}

MethodError: MethodError: no method matching iterate(::Type{ParticleCollection{RoombaState}})
Closest candidates are:
  iterate(!Matched::Core.SimpleVector) at essentials.jl:589
  iterate(!Matched::Core.SimpleVector, !Matched::Any) at essentials.jl:589
  iterate(!Matched::ExponentialBackOff) at error.jl:171
  ...

In [ ]:
function POMDPs.action(p::ToEnd, B::ParticleCollection{RoombaState})
    v = 1
    om = 1
    return RoombaAct(v, om)
end

## Create and Run the Environment

In [25]:
# first seed the environment
Random.seed!(2)

# reset the policy
p = ToEnd(0) # here, the argument sets the time-steps elapsed to 0

# run the simulation
c = @GtkCanvas()
win = GtkWindow(c, "Roomba Environment", 600, 600)
for (t, step) in enumerate(stepthrough(m, p, belief_updater, max_steps=100))
    @guarded draw(c) do widget
        
        # the following lines render the room, the particles, and the roomba
        ctx = getgc(c)
        set_source_rgb(ctx,1,1,1)
        paint(ctx)
        render(ctx, m, step)
        
        # render some information that can help with debugging
        # here, we render the time-step, the state, and the observation
        move_to(ctx,300,400)
        show_text(ctx, @sprintf("t=%d, state=%s, o=%.3f",t,string(step.s),step.o))
    end
    show(c)
    sleep(0.1) # to slow down the simulation
end

UndefVarError: UndefVarError: ToEnd not defined